In [1]:
from pprint import pprint
import json

from buffalo.algo.bpr import BPRMF
from buffalo.algo.options import BPRMFOption
from buffalo.data.mm import MatrixMarketOptions
from buffalo.misc import aux
from buffalo.misc import log
log.set_log_level(log.DEBUG)

### BPR Hyperparameter optimization

Buffalo exploits [hyperopt](https://github.com/hyperopt/hyperopt) to optimize hyperparameters

In [2]:
opt = BPRMFOption().get_default_option()
opt.num_workers = 6
opt.num_iters = 20
opt.evaluation_period = 20
opt.evaluation_on_learning = True

To use hyperparameter optimization, validation option must be set to evaluate the model during training.

In [3]:
data_opt = MatrixMarketOptions().get_default_option()
data_opt.input.main = 'data/ml-1m/main.mtx'
data_opt.input.iid = 'data/ml-1m/iid'
data_opt.input.uid = 'data/ml-1m/uid'
data_opt.data.path = '.5_hpopt.h5py'
data_opt.data.validation.p = 0.1
data_opt.data.validation.max_samples = 5000

### Set evaluation metric and validation dataset

you can see how the validation is going by using tensorboard

In [4]:
opt.validation = aux.Option({'topk': 10})
opt.tensorboard = aux.Option({'root' :'./bpr-validation', 'name': 'bpr-new'})

In [5]:
opt.optimize = aux.Option({
        'loss': 'val_ndcg',
        'max_trials':100,
        'deployment': True,
        'start_with_default_parameters': False,
        'space': {
                'num_negative_samples': ['randint', ['num_negative_samples', 1, 5]],
                'reg_u': ['uniform', ['reg_u', 0.01, 0.05]],
                'reg_i': ['uniform', ['reg_i', 0.01, 0.05]],
                'reg_j': ['uniform', ['reg_j', 0.01, 0.05]]
        }
    })

In [6]:
bpr = BPRMF(opt, data_opt=data_opt)
bpr.initialize()

[INFO    ] 2019-10-04 11:29:50 [mm.py:193] Create the database from matrix market file.
[DEBUG   ] 2019-10-04 11:29:50 [mm.py:198] Building meta part...
[PROGRESS] 100.00% 0.2/0.2secs 31.44it/s
[INFO    ] 2019-10-04 11:29:50 [mm.py:206] Creating working data...
[PROGRESS] 100.00% 2.6/2.6secs 4,233,314.61it/s
[DEBUG   ] 2019-10-04 11:29:54 [mm.py:210] Working data is created on /tmp/tmpgfansid0
[INFO    ] 2019-10-04 11:29:54 [mm.py:211] Building data part...
[INFO    ] 2019-10-04 11:29:54 [base.py:362] Building compressed triplets for rowwise...
[INFO    ] 2019-10-04 11:29:54 [base.py:363] Preprocessing...
[INFO    ] 2019-10-04 11:29:54 [base.py:366] In-memory Compressing ...
[INFO    ] 2019-10-04 11:29:54 [base.py:249] Load triplet files. Total job files: 7
[INFO    ] 2019-10-04 11:29:54 [base.py:396] Finished
[INFO    ] 2019-10-04 11:29:54 [base.py:362] Building compressed triplets for colwise...
[INFO    ] 2019-10-04 11:29:54 [base.py:363] Preprocessing...
[INFO    ] 2019-10-04 11:29

In [7]:
!mkdir models
bpr.opt.model_path = "models/bpr-best-model.bin"
bpr.optimize()

[optimizing... ] 0.00% 0.0/0.0secs 0.00it/s[INFO    ] 2019-10-04 11:30:14 [optimize.py:73] Found new best parameters: {'num_negative_samples': 2, 'reg_i': 0.037947079665678056, 'reg_j': 0.01492763307463982, 'reg_u': 0.0385737433261689} @ iter 1
[INFO    ] 2019-10-04 11:30:14 [optimize.py:79] Saving model... to models/bpr-best-model.bin
[optimizing... ] 1.00% 18.3/1831.4secs 0.05it/s[DEBUG   ] 2019-10-04 11:30:14 [optimize.py:84] Params({'num_negative_samples': 2, 'reg_i': 0.037947079665678056, 'reg_j': 0.01492763307463982, 'reg_u': 0.0385737433261689}) Losses({'train_loss': 0.0, 'val_ndcg': 0.04854122863019933, 'val_map': 0.031648377015489525, 'val_accuracy': 0.07621592337695639, 'val_rmse': 2.8708799838255823, 'val_error': 2.6341123029828073, 'eval_time': 1570156214.3249521, 'loss': 0.04854122863019933, 'status': 'ok'})
[INFO    ] 2019-10-04 11:30:37 [optimize.py:73] Found new best parameters: {'num_negative_samples': 4, 'reg_i': 0.03843104993570734, 'reg_j': 0.016811995147165984, 're

[optimizing... ] 14.00% 238.9/1706.7secs 0.06it/s[DEBUG   ] 2019-10-04 11:33:55 [optimize.py:84] Params({'num_negative_samples': 1, 'reg_i': 0.033883168181139685, 'reg_j': 0.0382788431813184, 'reg_u': 0.048138162464889764}) Losses({'train_loss': 0.0, 'val_ndcg': 0.04826808486545401, 'val_map': 0.03100333605189558, 'val_accuracy': 0.07700716277434848, 'val_rmse': 2.855877740910218, 'val_error': 2.62204476672411, 'eval_time': 1570156435.0569375, 'loss': 0.04826808486545401, 'status': 'ok'})
[optimizing... ] 15.00% 257.7/1718.2secs 0.06it/s[DEBUG   ] 2019-10-04 11:34:13 [optimize.py:84] Params({'num_negative_samples': 3, 'reg_i': 0.03625270535442547, 'reg_j': 0.014103082888534138, 'reg_u': 0.04395161736077109}) Losses({'train_loss': 0.0, 'val_ndcg': 0.047397635391807996, 'val_map': 0.03008370738682444, 'val_accuracy': 0.07635274692813619, 'val_rmse': 2.924877775219038, 'val_error': 2.6901355184674265, 'eval_time': 1570156453.848767, 'loss': 0.047397635391807996, 'status': 'ok'})
[optimizi

[optimizing... ] 29.00% 548.2/1890.2secs 0.05it/s[DEBUG   ] 2019-10-04 11:39:04 [optimize.py:84] Params({'num_negative_samples': 4, 'reg_i': 0.020749659367886773, 'reg_j': 0.024066913668558433, 'reg_u': 0.018737994518540566}) Losses({'train_loss': 0.0, 'val_ndcg': 0.04812807946884335, 'val_map': 0.031070200266459323, 'val_accuracy': 0.07662947783532664, 'val_rmse': 2.9747757414954323, 'val_error': 2.7421265095114706, 'eval_time': 1570156744.2710388, 'loss': 0.04812807946884335, 'status': 'ok'})
[optimizing... ] 30.00% 568.5/1894.9secs 0.05it/s[DEBUG   ] 2019-10-04 11:39:24 [optimize.py:84] Params({'num_negative_samples': 3, 'reg_i': 0.015582317767713008, 'reg_j': 0.03488844043902033, 'reg_u': 0.024691335268235436}) Losses({'train_loss': 0.0, 'val_ndcg': 0.04756272216943884, 'val_map': 0.030318167827820968, 'val_accuracy': 0.07679827557343696, 'val_rmse': 2.9245171428750334, 'val_error': 2.689729225540161, 'eval_time': 1570156764.591678, 'loss': 0.04756272216943884, 'status': 'ok'})
[op

[optimizing... ] 46.00% 867.9/1886.7secs 0.05it/s[DEBUG   ] 2019-10-04 11:44:23 [optimize.py:84] Params({'num_negative_samples': 1, 'reg_i': 0.02168712016040897, 'reg_j': 0.04255660964479449, 'reg_u': 0.036193206717950635}) Losses({'train_loss': 0.0, 'val_ndcg': 0.04844511867376781, 'val_map': 0.0315041687012823, 'val_accuracy': 0.07643747040822618, 'val_rmse': 2.855654207113664, 'val_error': 2.6218178948879243, 'eval_time': 1570157063.9781735, 'loss': 0.04844511867376781, 'status': 'ok'})
[optimizing... ] 47.00% 888.1/1889.6secs 0.05it/s[DEBUG   ] 2019-10-04 11:44:44 [optimize.py:84] Params({'num_negative_samples': 3, 'reg_i': 0.03600862711480236, 'reg_j': 0.018234925359684864, 'reg_u': 0.022459533846880344}) Losses({'train_loss': 0.0, 'val_ndcg': 0.047673689562484765, 'val_map': 0.030470005412244948, 'val_accuracy': 0.0768021709058549, 'val_rmse': 2.924780249098762, 'val_error': 2.6900230456471443, 'eval_time': 1570157084.2133822, 'loss': 0.047673689562484765, 'status': 'ok'})
[optim

[optimizing... ] 62.00% 1179.1/1901.8secs 0.05it/s[DEBUG   ] 2019-10-04 11:49:35 [optimize.py:84] Params({'num_negative_samples': 2, 'reg_i': 0.022783072562501568, 'reg_j': 0.03892057100877132, 'reg_u': 0.041114607463133876}) Losses({'train_loss': 0.0, 'val_ndcg': 0.04831050091337631, 'val_map': 0.03152193014185042, 'val_accuracy': 0.07559031676008506, 'val_rmse': 2.870754693765659, 'val_error': 2.63390600284338, 'eval_time': 1570157375.2022545, 'loss': 0.04831050091337631, 'status': 'ok'})
[optimizing... ] 63.00% 1202.4/1908.6secs 0.05it/s[DEBUG   ] 2019-10-04 11:49:58 [optimize.py:84] Params({'num_negative_samples': 4, 'reg_i': 0.037562014058192754, 'reg_j': 0.03813386389257914, 'reg_u': 0.04377996234162213}) Losses({'train_loss': 0.0, 'val_ndcg': 0.04855128899246194, 'val_map': 0.031131517551950535, 'val_accuracy': 0.07796930988157723, 'val_rmse': 2.975339829053369, 'val_error': 2.742764629125595, 'eval_time': 1570157398.5481827, 'loss': 0.04855128899246194, 'status': 'ok'})
[optimi

[optimizing... ] 79.00% 1524.4/1929.6secs 0.05it/s[DEBUG   ] 2019-10-04 11:55:20 [optimize.py:84] Params({'num_negative_samples': 3, 'reg_i': 0.020585599925194896, 'reg_j': 0.025194498121257097, 'reg_u': 0.037612637108029603}) Losses({'train_loss': 0.0, 'val_ndcg': 0.04723085026307073, 'val_map': 0.030163369584657103, 'val_accuracy': 0.07595217691115905, 'val_rmse': 2.9245297328847055, 'val_error': 2.6897382814645767, 'eval_time': 1570157720.4831424, 'loss': 0.04723085026307073, 'status': 'ok'})
[optimizing... ] 80.00% 1536.3/1920.4secs 0.05it/s[DEBUG   ] 2019-10-04 11:55:32 [optimize.py:84] Params({'num_negative_samples': 1, 'reg_i': 0.034256843235990056, 'reg_j': 0.035263405023110914, 'reg_u': 0.03970631450848239}) Losses({'train_loss': 0.0, 'val_ndcg': 0.04774068324917767, 'val_map': 0.030789420802713668, 'val_accuracy': 0.07563357118047584, 'val_rmse': 2.85585417469523, 'val_error': 2.6219527625799177, 'eval_time': 1570157732.4315143, 'loss': 0.04774068324917767, 'status': 'ok'})
[

[optimizing... ] 96.00% 1852.0/1929.1secs 0.05it/s[DEBUG   ] 2019-10-04 12:00:48 [optimize.py:84] Params({'num_negative_samples': 4, 'reg_i': 0.035902763065660474, 'reg_j': 0.028743615657162106, 'reg_u': 0.03924210177527224}) Losses({'train_loss': 0.0, 'val_ndcg': 0.047811610183556195, 'val_map': 0.03103042478754598, 'val_accuracy': 0.07535083496914106, 'val_rmse': 2.975333072866642, 'val_error': 2.7427518073558805, 'eval_time': 1570158048.0657687, 'loss': 0.047811610183556195, 'status': 'ok'})
[optimizing... ] 97.00% 1873.0/1931.0secs 0.05it/s[DEBUG   ] 2019-10-04 12:01:09 [optimize.py:84] Params({'num_negative_samples': 3, 'reg_i': 0.010458795687584926, 'reg_j': 0.0442140636846585, 'reg_u': 0.025613403778138172}) Losses({'train_loss': 0.0, 'val_ndcg': 0.04719021370410452, 'val_map': 0.0299987490881908, 'val_accuracy': 0.07634463165226552, 'val_rmse': 2.9245545107471904, 'val_error': 2.6898054398298266, 'eval_time': 1570158069.1429918, 'loss': 0.04719021370410452, 'status': 'ok'})
[op

{'trials': <hyperopt.base.Trials at 0x7eff20bf4a90>,
 'best': {'train_loss': 0.0,
  'val_ndcg': 0.04682917843070606,
  'val_map': 0.02983502784157934,
  'val_accuracy': 0.07521336219589161,
  'val_rmse': 2.924483305000444,
  'val_error': 2.68965353603363,
  'eval_time': 1570157295.1042256,
  'loss': 0.04682917843070606,
  'status': 'ok'},
 'best_parameters': {'num_negative_samples': 3,
  'reg_i': 0.022375571379744608,
  'reg_j': 0.04350461746145798,
  'reg_u': 0.044975905220846886}}

In [8]:
validation_res = bpr.get_optimization_data()

best parameter found is 

In [9]:
validation_res

{'trials': <hyperopt.base.Trials at 0x7eff20bf4a90>,
 'best': {'train_loss': 0.0,
  'val_ndcg': 0.04682917843070606,
  'val_map': 0.02983502784157934,
  'val_accuracy': 0.07521336219589161,
  'val_rmse': 2.924483305000444,
  'val_error': 2.68965353603363,
  'eval_time': 1570157295.1042256,
  'loss': 0.04682917843070606,
  'status': 'ok'},
 'best_parameters': {'num_negative_samples': 3,
  'reg_i': 0.022375571379744608,
  'reg_j': 0.04350461746145798,
  'reg_u': 0.044975905220846886}}